In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
from scipy.stats import norm
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

# --- Configuration ---
RISK_FREE_RATE = 0.045  # 4.5%
TRADING_DAYS = 252

# --- Helper Functions ---

def black_scholes_price(S, K, T, r, sigma, option_type='call'):
    """
    Calculate Black-Scholes option price.
    S: Spot price
    K: Strike price
    T: Time to maturity (in years)
    r: Risk-free rate
    sigma: Implied Volatility (decimal)
    option_type: 'call' or 'put'
    """
    if T <= 0:
        return max(0, S - K) if option_type == 'call' else max(0, K - S)
    
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    if option_type == 'call':
        price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    else:
        price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    
    return price

def get_historical_volatility(history, window=30):
    """Calculate annualized historical volatility."""
    returns = np.log(history['Close'] / history['Close'].shift(1))
    vol = returns.rolling(window=window).std() * np.sqrt(TRADING_DAYS)
    return vol

def fetch_data(ticker_symbol, start_date, end_date):
    """Fetch historical data from yfinance."""
    ticker = yf.Ticker(ticker_symbol)
    hist = ticker.history(start=start_date, end=end_date)
    return hist

# --- Strategy 1: Analyst Day Volatility Arbitrage ---
# Example: HOOD Dec 4, 2024 Investor Day

def run_analyst_day_strategy():
    print("\n--- Strategy 1: Analyst Day Volatility Arbitrage (HOOD Example) ---")
    ticker_symbol = "HOOD"
    event_date = "2024-12-04"
    
    # Setup dates
    event_dt = datetime.strptime(event_date, "%Y-%m-%d")
    entry_date = (event_dt - timedelta(days=7)).strftime("%Y-%m-%d") # Approx T-5 trading days
    exit_date = (event_dt + timedelta(days=1)).strftime("%Y-%m-%d")  # T+1
    
    print(f"Event: {ticker_symbol} Investor Day on {event_date}")
    print(f"Entry Date (T-5): {entry_date}")
    print(f"Exit Date (T+1): {exit_date}")

    # Fetch Data
    # Fetching a bit more buffer to calculate volatility
    data_start = (event_dt - timedelta(days=60)).strftime("%Y-%m-%d")
    data_end = (event_dt + timedelta(days=5)).strftime("%Y-%m-%d")
    
    hist = fetch_data(ticker_symbol, data_start, data_end)
    if hist.empty:
        print("Error: No data found for HOOD.")
        return

    # Calculate Historical Volatility for IV estimation
    hist['HistVol'] = get_historical_volatility(hist)
    
    # --- Entry Execution ---
    try:
        entry_row = hist.loc[entry_date]
    except KeyError:
        # Fallback to nearest date if exact date missing (weekend/holiday)
        entry_row = hist.asof(entry_date)
        
    spot_entry = entry_row['Close']
    # Strike: ATM or 1st OTM. Let's pick nearest whole number strike above spot
    strike = np.ceil(spot_entry) 
    
    # IV Assumption: Market underprices event. 
    # Realized vol might be high, but entry IV is "low" relative to event move?
    # Actually, the thesis is "Implied vol < historical realized vol for similar catalyst events"
    # Let's assume Entry IV is simply the 30-day historical vol at that moment.
    iv_entry = entry_row['HistVol']
    
    # Time to expiry: Strategy buys 2-week options? 
    # "Buy Dec 6 $36.50 calls" (Event Dec 4). Expiry Dec 6.
    expiry_date = "2024-12-06"
    expiry_dt = datetime.strptime(expiry_date, "%Y-%m-%d")
    entry_dt = datetime.strptime(entry_date, "%Y-%m-%d")
    
    dte_entry = (expiry_dt - entry_dt).days / 365.0
    
    call_price_entry = black_scholes_price(spot_entry, strike, dte_entry, RISK_FREE_RATE, iv_entry, 'call')
    
    print(f"Entry Spot: ${spot_entry:.2f}")
    print(f"Strike: ${strike:.2f}")
    print(f"Entry IV (est): {iv_entry:.2%}")
    print(f"Entry Call Price: ${call_price_entry:.2f}")

    # --- Exit Execution ---
    try:
        exit_row = hist.loc[exit_date]
    except KeyError:
        exit_row = hist.asof(exit_date)
        
    spot_exit = exit_row['Close']
    exit_dt = datetime.strptime(exit_date, "%Y-%m-%d")
    dte_exit = (expiry_dt - exit_dt).days / 365.0
    
    # IV Assumption at Exit: 
    # "Exit on vol collapse" usually applies to earnings. 
    # For Analyst Day, "Long gamma captures excess realized vol".
    # If the stock moved a lot, the option gains value.
    # We'll assume IV stays roughly same or drops slightly. Let's use same HistVol for simplicity, 
    # the P&L will come from Delta/Gamma (Spot move).
    iv_exit = iv_entry # Conservative assumption
    
    call_price_exit = black_scholes_price(spot_exit, strike, dte_exit, RISK_FREE_RATE, iv_exit, 'call')
    
    print(f"Exit Spot: ${spot_exit:.2f}")
    print(f"Exit Call Price: ${call_price_exit:.2f}")
    
    pnl = (call_price_exit - call_price_entry) / call_price_entry
    print(f"P&L: {pnl:.2%}")
    
    return {
        "Strategy": "Analyst Day",
        "Ticker": ticker_symbol,
        "Entry Date": entry_date,
        "Exit Date": exit_date,
        "Return": pnl
    }



# --- Strategy 2: Pre-Earnings Call Buying ---
# Mechanics: Buy 1st OTM calls T-2, Exit T+1.

def run_earnings_strategy(tickers=['BLK', 'MSFT', 'NVDA', 'AAPL']):
    print("\n--- Strategy 2: Pre-Earnings Call Buying ---")
    results = []
    
    for ticker_symbol in tickers:
        try:
            ticker = yf.Ticker(ticker_symbol)
            # Get earnings dates
            earnings = ticker.get_earnings_dates(limit=12)
            if earnings is None or earnings.empty:
                print(f"No earnings dates for {ticker_symbol}")
                continue
                
            # Filter for past earnings only to backtest
            # Filter for past earnings only to backtest
            # Fix timezone issue: Convert earnings index to naive or make now() aware
            now_aware = datetime.now().astimezone()
            past_earnings = earnings.index[earnings.index < now_aware]
            print(f"DEBUG: {ticker_symbol} - Total Earnings: {len(earnings)}, Past: {len(past_earnings)}")
            if len(past_earnings) > 0:
                print(f"DEBUG: First past earning: {past_earnings[0]}")
            
            # Take the last 2 earnings events for demonstration
            for event_dt in past_earnings[:2]:
                event_date = event_dt.strftime("%Y-%m-%d")
                
                # T-2 Entry
                entry_dt = event_dt - timedelta(days=4) # Buffer for weekends
                entry_date = entry_dt.strftime("%Y-%m-%d")
                
                # T+1 Exit
                exit_dt = event_dt + timedelta(days=1)
                exit_date = exit_dt.strftime("%Y-%m-%d")
                
                # Fetch Data
                data_start = (entry_dt - timedelta(days=60)).strftime("%Y-%m-%d")
                data_end = (exit_dt + timedelta(days=5)).strftime("%Y-%m-%d")
                print(f"DEBUG: Fetching data for {ticker_symbol} from {data_start} to {data_end}")
                hist = fetch_data(ticker_symbol, data_start, data_end)
                
                if hist.empty: 
                    continue
                
                # Fix timezone mismatch for asof lookup
                hist.index = hist.index.tz_localize(None)
                
                # Calculate Historical Volatility BEFORE row lookup
                hist['HistVol'] = get_historical_volatility(hist)
                
                # Find nearest trading days
                try:
                    entry_row = hist.asof(entry_date)
                    exit_row = hist.asof(exit_date)
                except:
                    continue
                    
                if pd.isna(entry_row['Close']) or pd.isna(exit_row['Close']):
                    continue

                spot_entry = entry_row['Close']
                strike = np.ceil(spot_entry * 1.02) # 1st OTM (approx 2% OTM)
                
                # IV Assumption: Pre-earnings IV is elevated. 
                # We'll use HistVol * 1.5 as a proxy for Pre-Earnings IV
                iv_entry = entry_row['HistVol'] * 1.5
                
                # Expiry: Assume nearest monthly or weekly, say 10 days out
                dte_entry = 10 / 365.0
                
                call_price_entry = black_scholes_price(spot_entry, strike, dte_entry, RISK_FREE_RATE, iv_entry, 'call')
                
                # Exit
                spot_exit = exit_row['Close']
                dte_exit = (10 - 3) / 365.0 # 3 days passed
                
                # IV Crush: IV drops back to normal (HistVol)
                iv_exit = exit_row['HistVol'] * 1.0 
                
                call_price_exit = black_scholes_price(spot_exit, strike, dte_exit, RISK_FREE_RATE, iv_exit, 'call')
                
                pnl = (call_price_exit - call_price_entry) / call_price_entry
                
                results.append({
                    "Strategy": "Earnings",
                    "Ticker": ticker_symbol,
                    "Event Date": event_date,
                    "Return": pnl
                })
                print(f"{ticker_symbol} Earnings {event_date}: {pnl:.2%}")
                
        except Exception as e:
            print(f"Error processing {ticker_symbol}: {e}")
            
    return results

# --- Strategy 3: Asymmetric Alpha (Systematic Covered Call) ---
# Mechanics: Short 10% OTM calls on Quality stocks.

def run_covered_call_strategy(tickers=['AAPL', 'MSFT', 'GOOGL', 'JNJ']):
    print("\n--- Strategy 3: Asymmetric Alpha (Covered Call) ---")
    results = []
    
    # Simulation period: Last 6 months
    start_date = (datetime.now() - timedelta(days=180)).strftime("%Y-%m-%d")
    end_date = datetime.now().strftime("%Y-%m-%d")
    
    for ticker_symbol in tickers:
        hist = fetch_data(ticker_symbol, start_date, end_date)
        hist['HistVol'] = get_historical_volatility(hist)
        
        # Monthly roll simulation
        # Buy stock, Sell 1M Call 10% OTM
        # Simplified: Check returns every 30 days
        
        dates = hist.index[::21] # Approx every month
        
        for i in range(len(dates)-1):
            entry_date = dates[i]
            exit_date = dates[i+1]
            
            spot_entry = hist.loc[entry_date]['Close']
            strike = spot_entry * 1.10 # 10% OTM
            
            iv_entry = hist.loc[entry_date]['HistVol']
            if pd.isna(iv_entry): iv_entry = 0.20 # Default
            
            dte = 30 / 365.0
            
            # Premium collected
            call_premium = black_scholes_price(spot_entry, strike, dte, RISK_FREE_RATE, iv_entry, 'call')
            
            # Exit (Settlement)
            spot_exit = hist.loc[exit_date]['Close']
            
            # Stock P&L
            stock_return = (spot_exit - spot_entry) / spot_entry
            
            # Option P&L (Short position)
            # Value at expiry (intrinsic only as we hold to expiry/roll)
            call_value_exit = max(0, spot_exit - strike)
            option_pnl = call_premium - call_value_exit
            
            # Total Position P&L (Covered Call)
            # (Stock Gain + Option Gain) / Initial Investment
            total_pnl = (spot_exit - spot_entry + option_pnl) / spot_entry
            
            results.append({
                "Strategy": "Covered Call",
                "Ticker": ticker_symbol,
                "Period": f"{entry_date.date()} to {exit_date.date()}",
                "Return": total_pnl
            })
            
    # Average return
    avg_ret = np.mean([r['Return'] for r in results])
    print(f"Average Monthly Return: {avg_ret:.2%}")
    return results

if __name__ == "__main__":
    results = []
    
    # Strategy 1
    res1 = run_analyst_day_strategy()
    if res1: results.append(res1)
    
    # Strategy 2
    res2 = run_earnings_strategy()
    results.extend(res2)
    
    # Strategy 3
    res3 = run_covered_call_strategy()
    results.extend(res3)
    
    # Summary
    print("\n" + "="*50)
    print("GOLDMAN SACHS STRATEGIES: BACKTEST REPORT")
    print("="*50)
    
    df_res = pd.DataFrame(results)
    
    # 1. Strategy Performance Summary
    summary = df_res.groupby("Strategy")['Return'].agg(['count', 'mean', 'min', 'max'])
    summary.columns = ['Trades', 'Avg Return', 'Min Return', 'Max Return']
    summary['Win Rate'] = df_res.groupby("Strategy")['Return'].apply(lambda x: (x > 0).mean())
    
    print("\n## 1. Performance Summary")
    print(summary.to_markdown(floatfmt=".2%"))
    
    # 2. Detailed Trade Log
    print("\n## 2. Trade Log")
    print(df_res[['Strategy', 'Ticker', 'Return']].to_markdown(index=False, floatfmt=".2%"))
    
    # 3. Replication Guide
    print("\n## 3. How to Replicate")
    print("""
1. **Install Dependencies**: `pip install yfinance pandas numpy scipy tabulate`
2. **Run Script**: `python3 gs_strategy_backtest.py`
3. **Data Source**: Uses `yfinance` (Yahoo Finance) for free historical data.
4. **Option Pricing**: Simulates prices using Black-Scholes model.
   - *Note*: Real-world results may vary due to actual Implied Volatility (IV) skew and bid-ask spreads.
    """)

    # Save to file for easy copy-pasting
    with open("backtest_report.md", "w") as f:
        f.write("# Goldman Sachs Strategies Backtest Report\n\n")
        f.write("## 1. Performance Summary\n")
        f.write(summary.to_markdown(floatfmt=".2%"))
        f.write("\n\n## 2. Trade Log\n")
        f.write(df_res[['Strategy', 'Ticker', 'Return']].to_markdown(index=False, floatfmt=".2%"))
        f.write("\n\n## 3. How to Replicate\n")
        f.write("""
1. **Install Dependencies**: `pip install yfinance pandas numpy scipy tabulate`
2. **Run Script**: `python3 gs_strategy_backtest.py`
3. **Data Source**: Uses `yfinance` (Yahoo Finance) for free historical data.
4. **Option Pricing**: Simulates prices using Black-Scholes model.
   - *Note*: Real-world results may vary due to actual Implied Volatility (IV) skew and bid-ask spreads.
        """)
    print("\n[+] Report saved to 'backtest_report.md'")




--- Strategy 1: Analyst Day Volatility Arbitrage (HOOD Example) ---
Event: HOOD Investor Day on 2024-12-04
Entry Date (T-5): 2024-11-27
Exit Date (T+1): 2024-12-05
Entry Spot: $37.65
Strike: $38.00
Entry IV (est): 87.39%
Entry Call Price: $1.92
Exit Spot: $38.92
Exit Call Price: $1.26
P&L: -34.36%

--- Strategy 2: Pre-Earnings Call Buying ---
DEBUG: BLK - Total Earnings: 12, Past: 12
DEBUG: First past earning: 2025-04-11 06:00:00-04:00
DEBUG: Fetching data for BLK from 2025-02-06 to 2025-04-17
BLK Earnings 2025-04-11: 110.46%
DEBUG: Fetching data for BLK from 2024-11-12 to 2025-01-21
BLK Earnings 2025-01-15: 134.69%
DEBUG: MSFT - Total Earnings: 12, Past: 12
DEBUG: First past earning: 2025-04-30 16:05:00-04:00
DEBUG: Fetching data for MSFT from 2025-02-25 to 2025-05-06
MSFT Earnings 2025-04-30: 124.63%
DEBUG: Fetching data for MSFT from 2024-11-26 to 2025-02-04
MSFT Earnings 2025-01-29: -97.91%
DEBUG: NVDA - Total Earnings: 12, Past: 12
DEBUG: First past earning: 2025-06-26 12:00:00-0